<a href="https://colab.research.google.com/github/TheAgaveFairy/MusicGeneration/blob/main/gpt2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://www.kaggle.com/code/arnavmishra6996/music-generation-using-decoder-only-model-gpt2

this is the inspiration, more or less copying it but in my own typing + modifications so i understand it

In [3]:
import numpy as np
import pandas as pd
import os
import transformers
from tqdm import tqdm

from helpers import list_files

In [4]:
MIDI_TEXT_FILENAME = 'midi_text_data.txt'
ABC_TEXT_FILENAME = 'all_abcs.txt'
MODEL_OUT_DIR = './modeloutput/'

In [5]:
def csv_loader(path):
    """
    Returns a clean Pandas DataFrame
    I just want to do clean up the API, do type forcing etc
    """
    df = pd.read_csv(test_file_path)
    #df = df.astype({"note_value": int, "velocity": int, "instrument_program_number": int})
    for c in df:
        print(c, df[c].dtype)
    return df


In [5]:
test_folder_path = './inputs/final/Debussy'
test_file_name = os.listdir(test_folder_path)[0]
test_file_path = os.path.join(test_folder_path, test_file_name)

#dataset = tf.data.Dataset.from_tensor_slices(os.listdir(test_folder_path))
#dataset

In [6]:
df = csv_loader(test_file_path)
print(df.head())

augment_data(df)
print(df.head())

start_time float64
end_time float64
note_value int64
velocity int64
instrument_program_number int64
   start_time   end_time  note_value  velocity  instrument_program_number
0    8.287847   8.444636          55        25                          0
1    8.287847   8.444636          53        21                          0
2    8.601425   8.758214          55        25                          0
3    8.601425   8.758214          53        21                          0
4   11.423627  11.580416          43        23                          0
HRT time: transing -2 steps
codeine time: tempo change 1.1472121509300008
bad musician: velocity -2
   start_time   end_time  note_value  velocity  instrument_program_number
0    9.507919   9.687789          53        23                          0
1    9.507919   9.687789          51        19                          0
2    9.867659  10.047530          53        23                          0
3    9.867659  10.047530          51        19              

GPT-2 needs this as text

In [3]:
def tokenize_midi(df, header, filename = None):
    """
    Takes in a DataFrame from a csv we made from a midi file, and turns it into GPT-2 friendly text.
    Saves it as a text file if desired and returns it as a String
    """
    
    representation = []
    representation.append(f'<|startofpiece|> {header}\n')

    for index, row in df.iterrows():
        text_line_items = []
        for name, values in row.items():
            if values.is_integer():
                text_line_items.append(f"{name}: {int(values)} ")
            else:
                text_line_items.append(f"{name}: {values} ")
            
        text_line_items[-1] = text_line_items[-1].strip()
        #text_line_items.append('\n')
        text_line = "".join(text_line_items) + "\n"
        #print(text_line)
        representation.append(text_line)
    representation.append(f'<|endofpiece|>\n')

    if filename:
        assert(filename.endswith('.txt'))
        with open(filename, 'w') as file:
            file.writelines(representation)
            
    return "".join(representation)
#a = tokenize_midi(df, "Composer: Debussy")

In [5]:
def prepareAllData():
    root_path = './inputs/final/'
    final_combined_data_path = os.path.join(root_path, MIDI_TEXT_FILENAME)
    with open(final_combined_data_path, "w") as file:
        file.write("<|startoftext|>\n")
        for root, dirs, files in os.walk(root_path, topdown=False):
            composer = root.split('/')[-1]
            for filename in tqdm(files, desc=f"Processing {composer:12}"):
                temp_path = os.path.join(root, filename)
                df = pd.read_csv(temp_path)
                file.write(tokenize_midi(df, f"Composer: {composer}"))
        file.write("<|endoftext|>\n")

#prepareAllData()

Processing Schubert    : 100%|██████████████| 1652/1652 [01:04<00:00, 25.67it/s]
Processing             : 0it [00:00, ?it/s]


Here we go

In [9]:
"""
import tensorflow as tf
from transformers import GPT2Tokenizer
from tqdm import tqdm

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

def load_tf_dataset(file_path, tokenizer, block_size=128, chunk_size=1000000):
    def chunk_generator():
        with open(file_path, 'r', encoding='utf-8') as f:
            while True:
                chunk = f.read(chunk_size)
                if not chunk:
                    break
                yield chunk

    def process_chunk(chunk):
        tokens = tokenizer.tokenize(chunk)
        ids = tokenizer.convert_tokens_to_ids(tokens)
        for i in range(0, len(ids) - block_size + 1, block_size):
            input_ids = ids[i:i+block_size]
            labels = ids[i+1:i+block_size+1] # shifted by 1
            labels = [-100 if token == tokenizer.pad_token_id else token for token in labels] # masking with -100

            yield {
                "input_ids": input_ids,#ids[i:i+block_size],
                "attention_mask": [1] * block_size,
                "labels": labels
            }

    def gen():
        for chunk in tqdm(chunk_generator(), desc="Processing chunks"):
            yield from process_chunk(chunk)

    return tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            {
                "input_ids": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "attention_mask": tf.TensorSpec(shape=(block_size,), dtype=tf.int32),
                "labels": tf.TensorSpec(shape=(block_size,), dtype=tf.int32)
            }
        )
    )

# Load the dataset
train_dataset = load_tf_dataset(MIDI_TEXT_FILENAME, tokenizer)
"""

'\nimport tensorflow as tf\nfrom transformers import GPT2Tokenizer\nfrom tqdm import tqdm\n\ntokenizer = GPT2Tokenizer.from_pretrained(\'gpt2\')\ntokenizer.pad_token = tokenizer.eos_token\n\ndef load_tf_dataset(file_path, tokenizer, block_size=128, chunk_size=1000000):\n    def chunk_generator():\n        with open(file_path, \'r\', encoding=\'utf-8\') as f:\n            while True:\n                chunk = f.read(chunk_size)\n                if not chunk:\n                    break\n                yield chunk\n\n    def process_chunk(chunk):\n        tokens = tokenizer.tokenize(chunk)\n        ids = tokenizer.convert_tokens_to_ids(tokens)\n        for i in range(0, len(ids) - block_size + 1, block_size):\n            input_ids = ids[i:i+block_size]\n            labels = ids[i+1:i+block_size+1] # shifted by 1\n            labels = [-100 if token == tokenizer.pad_token_id else token for token in labels] # masking with -100\n\n            yield {\n                "input_ids": input_ids,

In [10]:
#tf.config.list_physical_devices('CPU'),tf.config.list_physical_devices('GPU')

NameError: name 'tf' is not defined

In [ ]:
"""
#Claude 3.5Sonnet to the rescue, maybe
import tensorflow as tf
from transformers import GPT2Tokenizer, TFGPT2LMHeadModel
from transformers import DataCollatorForLanguageModeling, DefaultDataCollator
from tqdm import tqdm
import os

# Initialize tokenizer and data collator
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
"""
#data_collator = DataCollatorForLanguageModeling(
#    tokenizer=tokenizer,
#    mlm=True # changed from False
#)
"""
data_collator = DefaultDataCollator()

# Training configuration
class TrainingConfig:
    output_dir = "./gpt2-midi"
    num_train_epochs = 3
    per_device_train_batch_size = 4
    save_steps = 10_000
    save_total_limit = 2
    logging_dir = './logs'
    logging_steps = 500

config = TrainingConfig()

# Create output directory if it doesn't exist
os.makedirs(config.output_dir, exist_ok=True)
os.makedirs(config.logging_dir, exist_ok=True)

# Set up distributed training strategy
strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

# Create the model within the strategy scope
with strategy.scope():
    model = TFGPT2LMHeadModel.from_pretrained('gpt2')
    
    # Set up the optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5)
    
    # Compile the model
    model.compile(
        optimizer=optimizer,
        loss=model.compute_loss
    )

# Prepare the dataset
def prepare_dataset_for_training(dataset, batch_size):
    """Convert dataset to tf.data.Dataset format"""
    dataset = dataset.shuffle(1000)
    dataset = dataset.batch(batch_size, drop_remainder=True)
    """
    # Modify attention mask shape
    dataset = dataset.map(lambda x: {
        **x,
        "attention_mask": tf.reshape(x["attention_mask"], (tf.shape(x["attention_mask"])[0], 1, 1, tf.shape(x["attention_mask"])[2]))  # Reshape to (batch_size, 1, 1, seq_len)
    })
    """
    dataset = dataset.prefetch(tf.data.AUTOTUNE)
    return dataset

train_dataset = prepare_dataset_for_training(
    train_dataset,  # Your original train_dataset
    config.per_device_train_batch_size
)

# Custom callback for model saving
class ModelCheckpoint(tf.keras.callbacks.Callback):
    def __init__(self, save_steps, output_dir, save_total_limit):
        super().__init__()
        self.save_steps = save_steps
        self.output_dir = output_dir
        self.save_total_limit = save_total_limit
        self.step = 0

    def on_batch_end(self, batch, logs=None):
        self.step += 1
        if self.step % self.save_steps == 0:
            # Save the model
            checkpoint_dir = os.path.join(
                self.output_dir,
                f"checkpoint-{self.step}"
            )
            self.model.save_pretrained(checkpoint_dir)

            # Remove old checkpoints if exceeding limit
            checkpoints = sorted([
                d for d in os.listdir(self.output_dir)
                if d.startswith("checkpoint-")
            ])
            while len(checkpoints) > self.save_total_limit:
                checkpoint_to_remove = os.path.join(
                    self.output_dir,
                    checkpoints.pop(0)
                )
                tf.io.gfile.rmtree(checkpoint_to_remove)

# Set up callbacks
callbacks = [
    ModelCheckpoint(
        save_steps=config.save_steps,
        output_dir=config.output_dir,
        save_total_limit=config.save_total_limit
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=config.logging_dir,
        update_freq=config.logging_steps
    )
]

# Training
print("Starting training...")
history = model.fit(
    train_dataset,
    epochs=config.num_train_epochs,
    callbacks=callbacks
)

# Save the final model and tokenizer
model.save_pretrained(config.output_dir)
tokenizer.save_pretrained(config.output_dir)
print(f"Model saved to {config.output_dir}")
"""

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import transformers
import torch

transformers.utils.logging.set_verbosity(transformers.logging.WARNING)
transformers.utils.logging.enable_progress_bar

def fine_tune_gpt2(training_file, output_dir):
    os.makedirs(output_dir, exist_ok = True)
    # Load tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
    
    # Add special tokens if they're not already there
    special_tokens = {
        'additional_special_tokens': ['<|startofpiece|>', '<|endofpiece|>']
    }
    tokenizer.add_special_tokens(special_tokens)
    
    # Load model
    model = GPT2LMHeadModel.from_pretrained('gpt2', attn_implementation="flash_attention_2") #torch_dtype=torch.float16, 
    model.resize_token_embeddings(len(tokenizer))

    print("Model loaded")
    
    # Create dataset
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=training_file,
        block_size=256  # Adjust based on your data, was at 128
    )

    print("Dataset loaded")
    
    # Data collator
    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=False  # Not using masked language modeling
    )
    
    # Training arguments
    training_args = TrainingArguments(
        output_dir=output_dir,
        overwrite_output_dir=True,

        logging_dir="./logs",
        logging_steps=100,
        logging_first_step=True,
        logging_strategy="steps",
        
        num_train_epochs=3,
        per_device_train_batch_size=4,
        save_steps=1000,
        fp16=True,
        gradient_accumulation_steps=2,
        save_total_limit=1,
    )

    print("Device is", training_args.device)
    
    # Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        data_collator=data_collator,
        train_dataset=dataset,
    )

    print("Start Training")
    
    # Train
    trainer.train()    

    print("Training done, saving to", output_dir)
    
    # Save
    trainer.save_model()
    tokenizer.save_pretrained(output_dir)

In [7]:
fine_tune_gpt2(ABC_TEXT_FILENAME, './modeloutput/')

You are attempting to use Flash Attention 2.0 without specifying a torch dtype. This might lead to unexpected behaviour
You are attempting to use Flash Attention 2.0 with a model not initialized on GPU. Make sure to move the model to GPU after initializing it on CPU with `model.to('cuda')`.
The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
/home/paul-dutton/miniconda3/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


Model loaded
Dataset loaded
Device is cuda:0
Start Training


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
1,2.362000
100,1.404200
200,1.130700
300,1.090900
400,1.030900
500,1.011700
600,0.980500
700,0.962100
800,0.950300
900,0.936800


Training done, saving to ./modeloutput/


# Moving this to another file - getting outputs and converting to wav etc

In [19]:
def generate_music(model_path, prompt="Composer: Bach", max_length=512, num_return_sequences=1, temperature=1.0):
    """
    Generate music using your fine-tuned GPT-2 model
    
    Args:
        model_path: Path to the fine-tuned model
        prompt: Text prompt to start generation (e.g., "Composer: Bach")
        max_length: Maximum length of generated sequence
        num_return_sequences: Number of different sequences to generate
        temperature: Controls randomness (lower = more deterministic)
        
    Returns:
        List of generated MIDI objects
    """
    # Load model and tokenizer
    tokenizer = GPT2Tokenizer.from_pretrained(model_path)
    model = GPT2LMHeadModel.from_pretrained(model_path)
    
    # Format the prompt with the right structure
    formatted_prompt = f"<|startofpiece|>\n{prompt}\n"
    
    # Encode the prompt
    input_ids = tokenizer.encode(formatted_prompt, return_tensors="pt")
    
    # Generate continuation
    output_sequences = model.generate(
        input_ids=input_ids,
        max_length=max_length,
        temperature=temperature,
        top_k=50,
        top_p=0.95,
        num_return_sequences=num_return_sequences,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
    )
    
    # Decode and parse the output
    generated_sequences = []
    for sequence in output_sequences:
        text = tokenizer.decode(sequence, skip_special_tokens=False)
        
        # Make sure we stop at the end of piece token if present
        if "<|endofpiece|>" in text:
            text = text.split("<|endofpiece|>")[0] + "<|endofpiece|>"
            
        generated_sequences.append(text)
        
    # Convert text to MIDI
    midi_objects = []# [text_to_midi(seq) for seq in generated_sequences]
    
    return midi_objects, generated_sequences

midi_objects, generated_sequences = generate_music('./modeloutput/')

In [20]:
generated_sequences

['<|startofpiece|> \nComposer: Bach\nstart_time: 9.7 end_time: 9.795834 note_value: 76 velocity: 80 instrument_program_number: 40\nstart_time: 9.8 end_time: 9.895833 note_value: 74 velocity: 80 instrument_program_number: 40\nstart_time: 9.9 end_time: 10.045834 note_value: 73 velocity: 80 instrument_program_number: 40\nstart_time: 10.05 end_time: 10.195833 note_value: 71 velocity: 80 instrument_program_number: 40\nstart_time: 10.2 end_time: 10.295834 note_value: 70 velocity: 80 instrument_program_number: 40\nstart_time: 10.3 end_time: 10.395833 note_value: 68 velocity: 80 instrument_program_number: 40\nstart_time: 10.4 end_time: 10.495834 note_value: 67 velocity: 80 instrument_program_number: 40\nstart_time: 10.5 end_time: 10.795834 note_value: 69 velocity: 80 instrument_program_number: 40\nstart_time: 10.6 end_time: 10.795834 note_value: 66 velocity: 80 instrument_program_number: 40\nstart_time: 10.8 end_time: 10.99583334 note_value: 67 velocity: 80 instrument_program_number: 40\nstart

In [38]:
def text_to_midi(text):
    """Convert generated text back to MIDI format"""
    # Initialize a PrettyMIDI object
    midi = pretty_midi.PrettyMIDI()
    
    # Parse the text to extract notes
    lines = text.strip().split('\n')
    #print(lines)
    
    # Group notes by instrument
    instruments = {}
    
    # Extract all notes
    for line in lines:
        # Skip metadata and special tokens
        if line.startswith('<|') or not ('start_time:' in line and 'note_value:' in line):
            print(line, "skipped") 
            continue
            
        # Parse note information using regex
        start_match = re.search(r'start_time: ([0-9.]+)', line)
        end_match = re.search(r'end_time: ([0-9.]+)', line)
        note_match = re.search(r'note_value: (\d+)', line)
        vel_match = re.search(r'velocity: (\d+)', line)
        inst_match = re.search(r'instrument_program_number: (\d+)', line)
        
        if all([start_match, end_match, note_match, vel_match, inst_match]):
            start_time = float(start_match.group(1))
            end_time = float(end_match.group(1))
            pitch = int(note_match.group(1))
            velocity = int(vel_match.group(1))
            program = int(inst_match.group(1))

            print("got:", start_time, end_time, pitch, velocity, program)
            
            # Keep notes in valid MIDI range
            if 0 <= pitch <= 127 and velocity > 0:
                # Add instrument if needed
                if program not in instruments:
                    instruments[program] = pretty_midi.Instrument(program=program)
                    
                # Create a note
                note = pretty_midi.Note(
                    velocity=min(velocity, 127),  # Ensure valid velocity
                    pitch=pitch,
                    start=max(0, start_time),  # Ensure valid timing
                    end=max(start_time + 0.1, end_time)  # Ensure note has duration
                )
                
                instruments[program].notes.append(note)
    
    # Add instruments to the MIDI file
    for instrument in instruments.values():
        midi.instruments.append(instrument)
        
    return midi



In [43]:
import pretty_midi
import IPython.display
import re

test_midi = text_to_midi(generated_sequences[0])
test_midi = test_midi.synthesize()
#IPython.display.Audio(test_midi, rate=44100)

import pickle
# Pickling (serialization)
with open('data.pkl', 'wb') as file:
    pickle.dump(test_midi, file)

# Unpickling (deserialization)
with open('data.pkl', 'rb') as file:
    loaded_data = pickle.load(file)
IPython.display.Audio(loaded_data, rate=44100)

<|startofpiece|>  skipped
Composer: Bach skipped
got: 9.7 9.795834 76 80 40
got: 9.8 9.895833 74 80 40
got: 9.9 10.045834 73 80 40
got: 10.05 10.195833 71 80 40
got: 10.2 10.295834 70 80 40
got: 10.3 10.395833 68 80 40
got: 10.4 10.495834 67 80 40
got: 10.5 10.795834 69 80 40
got: 10.6 10.795834 66 80 40
got: 10.8 10.99583334 67 80 40
got: 11.0 11.195833 69 80 40
got: 12.0 12.39583334 70 80 40
got: 12.5 12.795834 71 80 40
got: 12.8 12.995834 72 80 40
got: 12.9 13.19583334 67 80 40
